In [1]:
import geemap
import ee
import os
from concurrent.futures.thread import ThreadPoolExecutor

In [ ]:
user_gee.sg_images()

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
roi = Map.draw_last_feature.geometry()
roi.getInfo()

{'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[-322.071846, 32.18956],
   [-322.071846, 44.851975],
   [-293.230449, 44.851975],
   [-293.230449, 32.18956],
   [-322.071846, 32.18956]]]}

In [ ]:
roi = geemap.shp_to_ee(r'E:/Data/clip/roi.shp')
Map.addLayer(roi, {}, 'roi')

In [ ]:
re = 10
n = 2
path = r'F:/r_factor/'
os.chdir(path)

In [ ]:
def re_value(img):
    n_id = img.get('system:id')
    img = img.clip(roi.geometry()).multiply(1000)
    mask = img.gt(re)
    return img.multiply(mask).set('system:id', n_id)

In [ ]:
re_value()

In [ ]:
images = ee.ImageCollection("ECMWF/ERA5/DAILY").filterDate('2019-01-01', '2020-07-06')

In [ ]:
images.first().getInfo()

In [ ]:
nemas = images.aggregate_array('system:index').getInfo()

In [ ]:
for year in range(2019, 2021):
    images = ee.ImageCollection("ECMWF/ERA5/DAILY")\
            .filterDate(str(year), str(year+1))

    names = images.aggregate_array('system:id').getInfo()
    
    def dow(i):
        start = i
        end = i+8
        if end > len(names):
            end = len(names)
        imgs = [ee.Image(names[j]) for j in range(start, end)]
        img = ee.ImageCollection(imgs).select('total_precipitation')\
                .map(re_value).sum()
        name = names[i].split('/')[-1]
        month = int(name) % 10000 // 100
        if (month >= 5) and (month <= 9):
            a = 0.3937
        else:
            a = 0.3101
        r_img = img.pow(1.7265).multiply(a).divide(n)
        geemap.ee_export_image(r_img, r'r_factor_{0}.tif'.format(name), 
                               scale=1000, crs='epsg:3857', region=roi.geometry())    
    
    length = images.size().getInfo()
    List = list(range(0, length, 8))
    with ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(dow, List)